In [1]:
import json
from pathlib import Path
from collections import defaultdict
from datetime import datetime, timedelta, timezone
from functools import partial

from timesignal import EventSignal, EventOrchestra

In [2]:
def load_json(p: Path):
    with p.open('r') as f:
        return json.load(f)

In [3]:
TEST_PATH = Path('log_json/lcmlog-2021-05-24.11.json')

test_events = load_json(TEST_PATH)

In [4]:
channels = set(ev['meta']['channel'] for ev in test_events)
channels

{'BOX_LEFT',
 'BOX_RIGHT',
 'BOX_STEREO',
 'BOX_STEREO_TRACK',
 'CAM-CFG-50-0503455191',
 'CAM-CFG-50-0503455192',
 'EXT_TARGET_3',
 'ML_CFG',
 'MWT_CONTROL_STAT',
 'MWT_SEARCH_STAT',
 'MWT_TARGET'}

In [5]:
events_by_channel = defaultdict(list)
for ev in test_events:
    events_by_channel[ev['meta']['channel']].append(ev)

In [6]:
chosen_channels = (
    'ML_CFG',
    'MWT_CONTROL_STAT',
    'MWT_SEARCH_STAT',
    'MWT_TARGET'
)

assert all(channel in channels for channel in chosen_channels)

In [7]:
for channel in chosen_channels:
    channel_events = events_by_channel[channel]
    print(channel)
    print(channel_events[0]['event'])
    print()

ML_CFG
{'score_threshold': 37, 'box_deviation': 10, 'reinit_threshold': 1, 'min_reinit_time': 250, 'max_epipolar_error': 5, 'model_name': 'docker run --gpus all -it --rm --network=host --shm-size=1g --ulimit memlock=-1 --ulimit stack=67108864 cvisionai/mbari_tracking:2020.08.11 python lcm_image_sub.py --image_channel $1 --bounding_box_channel $2 --image_width $3 --image_height $4 --model $5 --score_threshold=$6', 'target_class': 'bathochordaeus house', 'tracker_type': 'KCF', 'target_source': 'Best Available', 'box_combiner': 'Box Similarity', 'box_size_from_ml': 1, 'box_padding': 9, 'publish_mwt': 1, 'publish_ext': 1, 'box_size_x': 151, 'box_size_y': 178, 'rate_limit': 0.0, 'min_range': 0.3, 'max_range': 3.5, 'min_alt': -1.0, 'max_alt': 1.0, 'use_world_frame': 1, 'world_frame_timeout': 5}

MWT_CONTROL_STAT
{'header': {'publisher': 'tensorbook1:mwt_control(7017)', 'timestamp': 1621884313.5102735, 'sequence': 149942}, 'x_control': {'cmd': 0.6276600515990292, 'measure': 0.6160577517161535

In [8]:
tz = timezone(timedelta(hours=-8))

def get_datetime(ev: dict) -> datetime:
    return datetime.fromtimestamp(ev['meta']['timestamp'] / 1e6, tz=tz).astimezone(tz=timezone.utc)

In [13]:
def event_to_value(ev, indices) -> tuple:
    v = ev['event']
    try:
        for index in indices:
            v = v[index]
        v = float(v)
    except:
        v = None
    return get_datetime(ev), v

In [10]:
def signal_from_events(events, indices, interpolation='nearest') -> EventSignal:
    return EventSignal(map(partial(event_to_value, indices=indices), events), interpolation=interpolation)

In [11]:
mwt_search_stat_events = events_by_channel['MWT_SEARCH_STAT']
mwt_search_stat_orchestra = EventOrchestra({
    'x_mode': signal_from_events(mwt_search_stat_events, ('x_mode',)),
    'x_effort_cmd': signal_from_events(mwt_search_stat_events, ('x_effort_cmd',)),
    'y_mode': signal_from_events(mwt_search_stat_events, ('y_mode',)),
    'y_effort_cmd': signal_from_events(mwt_search_stat_events, ('y_effort_cmd',)),
    'z_mode': signal_from_events(mwt_search_stat_events, ('z_mode',)),
    'z_effort_cmd': signal_from_events(mwt_search_stat_events, ('z_effort_cmd',))
})

In [37]:
mwt_target_events = events_by_channel['MWT_TARGET']
mwt_target_orchestra = EventOrchestra({
    'range_meters': signal_from_events(mwt_target_events, ('range_meters',), interpolation='quadratic'),
    'range_valid': signal_from_events(mwt_target_events, ('range_valid',)),
    'bearing_degrees': signal_from_events(mwt_target_events, ('bearing_degrees',), interpolation='quadratic'),
    'bearing_valid': signal_from_events(mwt_target_events, ('bearing_valid',)),
    'z_meters': signal_from_events(mwt_target_events, ('z_meters',), interpolation='quadratic'),
    'z_valid': signal_from_events(mwt_target_events, ('z_valid',)),
})

In [38]:
root_orchestra = EventOrchestra({
    'mwt_search_stat': mwt_search_stat_orchestra,
    'mwt_target': mwt_target_orchestra,
})

In [75]:
import random
from timeit import timeit

N = 1000
elapsed_time = timeit(lambda: root_orchestra[root_orchestra.start + timedelta(milliseconds=random.randint(0, 10000))], number=N)

print(round(elapsed_time / N * 1e3, 2), 'ms')

0.42 ms
